# Benchmarking Code and Plots

In [1]:
import os
import importlib
from PIL import Image
from tqdm import tqdm
import test_utils
 # replace with actual import if needed
importlib.reload(test_utils)
from test_utils import calculate_metrics_batch 

/data/maryam.sana/anaconda3/envs/unicontrol/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
/home/maryam.sana/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /data/maryam.sana/anaconda3/envs/unicontrol/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth


/data/maryam.sana/anaconda3/envs/unicontrol/lib/python3.8/site-packages/lpips/lpips.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(mod

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /data/maryam.sana/anaconda3/envs/unicontrol/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth


/data/maryam.sana/anaconda3/envs/unicontrol/lib/python3.8/site-packages/lpips/lpips.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(mod

In [3]:
# === Paths ===
base_pred_dir = "./test_codec"
base_orig_dir = "../data/UVG_test_data"
output_file = "metrics_results_h265.txt"

# Open output file and write header
with open(output_file, "w") as fout:
    fout.write("Video,CRF,PSNR,MS-SSIM,LPIPS,FID,FVD\n")

    # Loop over videos with progress bar
    for video in tqdm(sorted(os.listdir(base_pred_dir)), desc="Videos"):
        h264_dir = os.path.join(base_pred_dir, video, "h265")
        orig_img_dir = os.path.join(base_orig_dir, video, "1080p")

        if not os.path.isdir(h264_dir) or not os.path.isdir(orig_img_dir):
            continue

        # Sort and load original frames once per video
        orig_files = sorted([f for f in os.listdir(orig_img_dir) if f.endswith(".png")])
        orig_frames = [Image.open(os.path.join(orig_img_dir, f)) for f in orig_files]

        # Loop over each CRF folder with inner progress bar
        for crf_folder in tqdm(sorted(os.listdir(h264_dir)), desc=f"{video} CRFs", leave=False):
            crf_path = os.path.join(h264_dir, crf_folder)
            if not os.path.isdir(crf_path):
                continue

            # Sort and load predicted frames
            pred_files = sorted([f for f in os.listdir(crf_path) if f.endswith(".png")])
            pred_frames = [Image.open(os.path.join(crf_path, f)) for f in pred_files]

            # Match frame counts
            min_len = min(len(orig_frames), len(pred_frames))
            metrics = calculate_metrics_batch(orig_frames[:min_len], pred_frames[:min_len])

            # Write metrics to file
            fout.write(
                f"{video},{crf_folder},"
                f"{metrics['PSNR']:.4f},{metrics['MS-SSIM']:.4f},"
                f"{metrics['LPIPS']:.4f},{metrics['FID']:.4f},{metrics['FVD']:.4f}\n"
            )
            fout.flush()

print(f"\n✅ Metrics saved to {output_file}")


Videos: 100%|█████████████████████████████████████████| 7/7 [06:12<00:00, 53.20s/it]


✅ Metrics saved to metrics_results_h265.txt
